In [1]:
import os

In [2]:
os.getcwd()

'/Users/vishalsharma/Downloads/Coccidiosis Chicken Disease Classifier/research'

In [3]:
os.chdir("../")

In [4]:
from src.coccidiosis_chicken_disease_classifier.constants import *
from src.coccidiosis_chicken_disease_classifier.utils.common import read_yaml, create_directories
from dataclasses import dataclass
from pathlib import Path

import zipfile
from urllib.request import urlretrieve
from src.coccidiosis_chicken_disease_classifier import logger
from src.coccidiosis_chicken_disease_classifier.utils.common import get_size

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzipped_data_dir: Path



In [5]:
class ConfigManager:
    def __init__(self, config_filepath: Path = CONFIG_FILE_PATH, params_filepath: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories(path= Path(self.config.artifacts_root))

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories(path= Path(config.root_dir))

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzipped_data_dir= config.unzipped_data_dir
        )

        return data_ingestion_config

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urlretrieve(
                url=self.config.source_URL, filename=self.config.local_data_file
            )
            logger.info(
                f"{filename} downloaded successfully and saved in {self.config.local_data_file}"
            )
        else:
            logger.info(
                f"File already exists in {self.config.local_data_file} with size {get_size(file_path=Path(self.config.local_data_file))}"
            )

    def extract_zip_file(self):
        unzip_filepath = self.config.unzipped_data_dir
        print(unzip_filepath,self.config.local_data_file,)
        os.makedirs(unzip_filepath, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_file:
            zip_file.extractall(unzip_filepath)

In [8]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion_obj = DataIngestion(config=data_ingestion_config)

    data_ingestion_obj.download_file()
    data_ingestion_obj.extract_zip_file()

except Exception as e:
    raise e


2025-12-04 15:34:33,185 - INFO - common:21 - YAML file: config/config.yaml loaded successfully
2025-12-04 15:34:33,187 - INFO - common:21 - YAML file: params.yaml loaded successfully
2025-12-04 15:34:33,188 - INFO - common:34 - Created directory at: artifacts
2025-12-04 15:34:33,188 - INFO - common:34 - Created directory at: artifacts/data_ingestion
2025-12-04 15:34:33,189 - INFO - 2675924837:14 - File already exists in artifacts/data_ingestion/chicken_fecal_images.zip with size 11345 kb
artifacts/data_ingestion artifacts/data_ingestion/chicken_fecal_images.zip
